In [1]:
import os

from docarray import Document, DocumentArray
from fastapi import APIRouter
from tqdm import tqdm

from routers.processor_service import ProcessorService
from routers.processor_router import preprocess_documents

from gensim import corpora, models

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import gensim
import nltk

import spacy

In [4]:
result_preprocess = preprocess_documents()
    
text = result_preprocess["results"][1]["texto"]

def make_bigrams(texts):
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    trigram = gensim.models.Phrases(bigram[texts], threshold=100)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


In [5]:
data_word_bigrams = make_bigrams(text)


In [9]:

tokenized_text = nltk.word_tokenize(text)
    
dictionary = corpora.Dictionary([tokenized_text])
corpus = [dictionary.doc2bow(tokenized_text) for text in tokenized_text]

In [10]:
print(corpus[:1][0][:30])


[(0, 4), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 6), (8, 4), (9, 2), (10, 2), (11, 14), (12, 2), (13, 4), (14, 2), (15, 2), (16, 2), (17, 2), (18, 2), (19, 2), (20, 2), (21, 2), (22, 14), (23, 10), (24, 2), (25, 2), (26, 2), (27, 2), (28, 10), (29, 8)]


In [11]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=dictionary,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [14]:
from pprint import pprint

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.020*"interna" + 0.020*"conselho" + 0.020*"constituinte" + 0.017*"cada" + '
  '0.016*"representantes" + 0.015*"assembléia" + 0.014*"resolução" + '
  '0.014*"delegados" + 0.012*"campus" + 0.011*"desta"'),
 (1,
  '0.028*"constituinte" + 0.023*"interna" + 0.020*"cada" + 0.018*"delegados" + '
  '0.017*"conselho" + 0.017*"assembléia" + 0.016*"representantes" + '
  '0.016*"campus" + 0.015*"resolução" + 0.013*"categoria"'),
 (2,
  '0.024*"interna" + 0.021*"assembléia" + 0.020*"representantes" + '
  '0.019*"constituinte" + 0.017*"resolução" + 0.016*"cada" + 0.016*"conselho" '
  '+ 0.014*"delegados" + 0.014*"categoria" + 0.012*"tocantins"'),
 (3,
  '0.025*"interna" + 0.023*"constituinte" + 0.020*"conselho" + 0.019*"cada" + '
  '0.017*"representantes" + 0.017*"assembléia" + 0.017*"delegados" + '
  '0.015*"resolução" + 0.014*"campus" + 0.013*"dia"'),
 (4,
  '0.021*"interna" + 0.021*"conselho" + 0.018*"cada" + 0.018*"assembléia" + '
  '0.017*"representantes" + 0.015*"delegados" + 0.015*"c

In [27]:
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
# Compute Coherence Score
lda_model = LdaModel(corpus, id2word=dictionary, num_topics=20, alpha=0.1, passes=10)

pprint(lda_model.print_topics())



[(0,
  '0.024*"interna" + 0.021*"assembléia" + 0.018*"constituinte" + '
  '0.018*"conselho" + 0.016*"cada" + 0.016*"representantes" + '
  '0.015*"categoria" + 0.013*"federal" + 0.012*"presente" + '
  '0.012*"funcionamento"'),
 (1,
  '0.026*"constituinte" + 0.023*"interna" + 0.022*"conselho" + '
  '0.020*"assembléia" + 0.019*"resolução" + 0.018*"cada" + '
  '0.017*"representantes" + 0.015*"delegados" + 0.013*"categoria" + '
  '0.013*"federal"'),
 (2,
  '0.029*"conselho" + 0.022*"delegados" + 0.020*"assembléia" + 0.019*"interna" '
  '+ 0.018*"constituinte" + 0.018*"cada" + 0.015*"representantes" + '
  '0.012*"funcionamento" + 0.012*"presente" + 0.012*"resolução"'),
 (3,
  '0.020*"interna" + 0.019*"constituinte" + 0.018*"cada" + 0.017*"assembléia" '
  '+ 0.016*"resolução" + 0.016*"conselho" + 0.014*"delegados" + '
  '0.013*"representantes" + 0.012*"eleição" + 0.012*"categoria"'),
 (4,
  '0.035*"interna" + 0.021*"cada" + 0.018*"constituinte" + 0.016*"conselho" + '
  '0.014*"representantes"

c:\Users\Aline\anaconda3\envs\pre_env\Lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in scalar divide
  m_lr_i = np.log(numerator / denominator)
c:\Users\Aline\anaconda3\envs\pre_env\Lib\site-packages\gensim\topic_coherence\indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in scalar divide
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


Coherence Score:  nan


In [29]:
import nltk

tokenized_text = nltk.word_tokenize(text)


coherence_model_lda = CoherenceModel(model=lda_model, texts=[tokenized_text], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.35669589445084465


In [19]:
print(text)
print(dictionary)

serviço público federal fundação  resolução conselho universitário n  dispõe sobre normas funcionamento assembléia constituinte interna  egrégio conselho universitário universidade f ederal tocantins reunido sessão dia abril resolve   assembléia constituinte interna universidade f ederal tocantins autônoma entendo-se delegados representantes categorias docente discente técnico-administrativa eleitos dir eta livremente pares através voto secreto   projeto final estatuto regimento geral un iversidade federal tocantins submetido conselho univers itário aprovação  atividades assembléia constituinte inte rna precedem todas demais   constituinte interna deflagrada inicia lmente cada campus garantindo picipação plena comunidade universitária  atividades iniciais constituinte intern deverão desenvolvidas prazo meses pir data aprovação presente resolução  cada campus deverá organizar especificidade s realizar debates internos desta primeira etapa  dia junho proceder-se-á el eição delegados cate